In [15]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("gtfs_tag.db")
c = conn.cursor()

$\to$ télécharger les données depuis :  
https://www.data.gouv.fr/fr/datasets/horaires-theoriques-du-reseau-tag/  
  
$\to$ Dans le dossier "./SEM-GTFS"

# Importer agency et stops avec Pandas

In [7]:
agency = pd.read_csv("SEM-GTFS/agency.txt")
agency

,agency_id,agency_name,agency_url,agency_timezone,agency_lang,agency_phone
0,SEM,Mobilités M - Tag,https://www.mobilites-m.fr/,Europe/Paris,FR,438703870


In [8]:
stops = pd.read_csv("SEM-GTFS/stops.txt")
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,zone_id,location_type,parent_station,wheelchair_boarding
0,0004,0004,"Meylan, Revirée - Sayettes",45.21220,5.76728,NaN,NaN,REVIREES,0
1,0006,0006,"La Tronche, Commandant Nal",45.20323,5.74535,NaN,NaN,COMMANAL,1
2,0007,0007,"La Tronche, Musée Hébert",45.20564,5.74888,NaN,NaN,MUSHEBER,1
3,0008,0008,"La Tronche, Centre Théologique",45.20617,5.75214,NaN,NaN,CENTHEOL,1
4,0011,0011,"Meylan, Malacher",45.21128,5.78724,NaN,NaN,MALACHER,1


In [13]:
agency.loc[:,["agency_id", "agency_name"]].to_sql("agency", con=conn, if_exists = "append", index= False)

In [14]:
stops.loc[:, ["stop_id", "stop_name", "stop_lat", 'stop_lon']].to_sql("Stops", con = conn, if_exists = "append", index = False)

# Bulk Insert :

In [85]:
# Étape 3.4.1 - implémenter une fonction qui génère une commande insertion SQL

def gen_insert_query(table_name, a_dict):
    cols = tuple(a_dict.keys())
    vals = tuple(a_dict.values())
    return f"INSERT INTO {table_name} {cols} VALUES {vals}"

In [107]:
## test de la fonction :

dicotest = {"stop_id" : 9999, "stop_name":"Ceci est un nom d'arrêt", "stop_lat":45.21220, 'stop_lon':5.75214}


print(gen_insert_query("table_name", dicotest))

INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES (9999, "Ceci est un nom d'arrêt", 45.2122, 5.75214)


In [113]:
# Étape 3.4.2 - implémenter une fonction qui génère des commandes d’insertion SQL

def get_insert_queries(table_name, df):
    querylist = []
    for row in range (len(df)):
        querylist.append(gen_insert_query(table_name, dict(df.iloc[row,:])))
    return querylist

In [114]:
## Test de la fonction : 

print(get_insert_queries("table_name", stops[["stop_id", "stop_name", "stop_lat", 'stop_lon']]))

["INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0004', 'Meylan, Revirée - Sayettes', 45.2122, 5.76728)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0006', 'La Tronche, Commandant Nal', 45.20323, 5.74535)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0007', 'La Tronche, Musée Hébert', 45.20564, 5.74888)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0008', 'La Tronche, Centre Théologique', 45.20617, 5.75214)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0011', 'Meylan, Malacher', 45.21128, 5.78724)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0012', 'La Tronche, La Carronnerie Basse', 45.20094, 5.75814)", "INSERT INTO table_name ('stop_id', 'stop_name', 'stop_lat', 'stop_lon') VALUES ('0015', 'Meylan, Granier', 45.21024, 5.78405)", "INSERT INTO table_name ('stop_id

In [183]:
# Étape 3.4.3 - implémenter une procédure qui crée un fichier SQL

def gen_insert_file(filename, table_name, df):
    filewriter = open(f"./Test/{filename}", "a")
    #filewriter.write("BEGIN TRANSACTION import_des_data; ")
    #filewriter.write(f"CREATE TABLE {table_name} {tuple(df.columns)}")
    liste = get_insert_queries(table_name, df)
    for line in range(len(liste)):
        filewriter.write(f"\n{liste[line]};")
    #filewriter.write("\nCOMMIT TRANSACTION import_des_data;")                
    filewriter.close()

In [161]:
# trips
trips = pd.read_csv("SEM-GTFS/trips.txt")
#trips = trips["trip_id", "trip_short_name", "route_id"]

# stoptimes
stoptimes = pd.read_csv("SEM-GTFS/stop_times.txt")
#stoptimes = stoptimes["trip_id", "stop_id", "arrival_time", "departure_time", "stop_sequence"]

In [151]:
gen_insert_file("Stops.sql", "Stops", stops)

In [184]:
gen_insert_file("Trips.sql", "Trips", trips)
gen_insert_file("Stop_Times.sql", "Stop_Times", stoptimes)